In [1]:
import onnx
import torch
from pathlib import Path
from onnxruntime_extensions import pnp, OrtPyFunction
from transformers import AutoTokenizer, GPT2LMHeadModel
from transformers.onnx import export, FeaturesManager
import onnxruntime as _ort
from onnxruntime_extensions import get_library_path as _lib_path
import numpy as np
so = _ort.SessionOptions()
so.enable_profiling = True
so.register_custom_ops_library(_lib_path())

In [2]:
model_name = "textattack/bert-base-uncased-MRPC"

tokenizer = AutoTokenizer.from_pretrained(model_name)
mrpc_text1 = "The quick brown fox jumps over the lazy dog"
mrpc_text2 = "A fast brown fox leaps over a lazy dog"
dummy_input = tokenizer(mrpc_text1,
                        mrpc_text2,  # MRPC 是句子对任务
                        max_length=128,  # 设置合适的最大长度
                        truncation=True,  # 启用截断
                        padding='max_length',  # 启用填充到最大长度

                        );
# dict(dummy_input)
# for key in dummy_input.keys():
#     dummy_input[key] = dummy_input[key].tolist()
dummy_input = dict(dummy_input)
dummy_input.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [3]:
# 加载onnx模型，查看输入输出名字
model_path = r"D:\project\inference_controller\tets\model_0.onnx"
model = onnx.load(model_path)
# 获取模型图
graph = model.graph
# 打印输入名称
print("Model Inputs:")
for input in graph.input:
    print(input.name)

# 打印输出名称
print("Model Outputs:")
for output in graph.output:
    print(output.name)

Model Inputs:
input
Model Outputs:
output
172


In [6]:
dummy_input = np.zeros((1, 3,32,32), dtype=np.float32)
session = _ort.InferenceSession(model_path, so)
re = session.run(None, {"input": dummy_input})
profile_file = session.end_profiling()

In [7]:
profile_file

'onnxruntime_profile__2024-08-31_19-13-38.json'